In [1]:
# A person class for spoofing Google

# your phone tracks:
# steps
# heartrate
# current location
# activity type: sleep, wake, walking, driving

# it can make inferences about calories burned, 
# do address place lookups, and current establishments

# location is updated > 14x/hour
# ad domains
# camera
# microphone
# call log
# SMS messages

> Since the spoofer is kind of like the Other's "heart" and "eyes" in terms of the data it produces, the idea of it existing in the gallery is kind of spooky, in a good way!  Sort of like physical proof of the coldness and inaccuracy of the data collected.  I like how the first image looks very technical.  The fact that it's generating quantitative versions of "emotions" makes that really funny.  

> I've also been thinking about how this project might exist online as well.  Since it is a simulated person, having feeds for the "eyes" (the Google Earth images", the "heart" (moods/sleep/BPM), and whatever may correlate to the Amazon searches ("brain", maybe?) could be interesting as well!  

In [13]:
import time
import logging
import json

from shapely.geometry import Point
from shapely.wkt import dumps, loads
from peewee import *

In [14]:
DEBUG = True
logger = logging.getLogger(__name__)

if DEBUG:
    db = SqliteDatabase(':memory:')
else:
    db = SqliteDatabase('other.db')

In [15]:
class BaseModel(Model):
    class Meta:
        database = db

class Person(BaseModel):
    name = CharField()

    def get_mytimepoints(self):
        return self.timepoints

class Brain(BaseModel):
    actor = ForeignKeyField(Person, backref='shoppingurls')
    timestamp = TimestampField()
    urls = []
    
class Eyes(BaseModel):
    actor = ForeignKeyField(Person, backref='streetviews')
    timestamp = TimestampField()

    streetviews = []
    
class Heart(BaseModel):
    actor = ForeignKeyField(Person, backref='heartbeats')
    timestamp = TimestampField()

    heartrate = []
    
class Legs(BaseModel):
    actor = ForeignKeyField(Person, backref='steps')
    timestamp = TimestampField()

    steps = []
    
class Timepoint(BaseModel):
    actor = ForeignKeyField(Person, backref='timepoints')
    timestamp = TimestampField()
    point = CharField()
    mode = CharField()

    def __repr__(self):
        return self.timestamp, self.mode, loads(self.point)


class Step(BaseModel):
    # do we count steps individually
    # or count them in a 24 hour period?
    actor = ForeignKeyField(Person, backref='steps')
    steps = IntegerField()
    timestamp = TimestampField()

In [5]:
# Connect to our database.
# db.connect()

# Create the tables.
db.create_tables([Person,Timepoint,Step])

In [6]:
other = Person.create(name='OTHER')
other.save()

1

In [7]:
unionstation = Point(34.0562, -118.2365)
Timepoint.insert(actor=1, point=dumps(unionstation), mode="WALK").execute()

1

In [8]:
print(other.__repr__())

<Person: 1>


In [9]:
avenel = Point(34.109950,-118.269000)
Timepoint.insert(actor=1, point=dumps(avenel), mode="WALK").execute()

2

In [10]:
for tp in Person.get_by_id(1).timepoints:
    print(tp.actor,loads(tp.point),tp.timestamp,tp.mode)

1 POINT (34.0562 -118.2365) 2019-06-03 13:25:01 WALK
1 POINT (34.10995 -118.269) 2019-06-03 13:25:04 WALK


In [11]:
Step.insert(actor=1, steps=1).execute()

1

In [12]:
for tp in Person.get_by_id(1).steps:
    print(tp.actor,tp.steps,tp.timestamp)

1 1 2019-06-03 13:25:05


In [17]:
import simplejson as json

In [20]:
heartratedata = "../data/heartrate.json"
# this is 24 hours of heart-rate sampled on average every 2.76 seconds
# time has the format "19:14:00" "%H:%m:%S"
# appears to be UTC/GMT

# [{
#   "time" : "08:00:07",
#   "value" : {
#     "bpm" : 78,
#     "confidence" : 2
#   }
# }

sleepdata = "../data/sleep.json"


stepdata = "../data/steps.json"
# datetime has the format "Tue 19:14:00" "%a %H:%m:%S"
# appears to be UTC/GMT

# [{
#   "dateTime" : "Sat 08:03:00",
#   "value" : "0"
# },

In [36]:
with open(heartratedata, 'r') as f:
    json_text = f.read()

In [37]:
heartrate_list = json.loads(json_text)
# print(heartrate_dict['explanation'])

In [40]:
heartrate_list[10]['time'],heartrate_list[10]['value']['bpm']

('08:02:02', 79)

In [34]:
heartrate_list[10]['value']['bpm']

79

In [47]:
from datetime import datetime

now = datetime.now()
seconds_since_midnight = (now - now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()

print( int(seconds_since_midnight) )

55219


In [50]:

d = time.strptime("08:02:02", "%H:%m:%S")
print(d)

time.struct_time(tm_year=1900, tm_mon=2, tm_mday=1, tm_hour=8, tm_min=0, tm_sec=2, tm_wday=3, tm_yday=32, tm_isdst=-1)


In [55]:
d = datetime.strptime("08:02:02", "%H:%m:%S").second
print (d)

2


In [97]:
def gimmeSeconds(thetime="08:02:02", thefmt="%H:%M:%S"):
    s = datetime.strptime(thetime, thefmt).second * 1
    ms = datetime.strptime(thetime, thefmt).minute * 60
    hs = datetime.strptime(thetime, thefmt).hour * 360
#     print(s,ms,hs,datetime.strptime("01:13:12", thefmt).minute)
    return int(s+ms+hs)

In [100]:
gimmeSeconds()

3002